# Deploy an Embedding Model on SAP Generative AI Hub

## Prerequisites
- SAP BTP Enterprise Account
- SAP AI Core with "Extended" [Service Plan](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/service-plans)
- SAP AI Launchpad with "Standard" [Service Plan](https://help.sap.com/docs/ai-launchpad/sap-ai-launchpad/service-plans)

Run the code cell below to install SAP Generative AI Hub SDK, if it's missing.

In [1]:
try:
    from gen_ai_hub.orchestration.service import OrchestrationService
    %pip show generative-ai-hub-sdk
except:
    ## See details at https://pypi.org/project/generative-ai-hub-sdk/
    # To install this SDK, use the following pip command, which includes support for all models including langchain support:
    %pip install generative-ai-hub-sdk[all]
    # The default installation only includes OpenAI models (without langchain support):
    # %pip install generative-ai-hub-sdk
    # You can install a subset of the extra libraries (without langchain support) by specifying them in square brackets:
    # pip install "generative-ai-hub-sdk[google, amazon]"

Name: generative-ai-hub-sdk
Version: 3.8.0
Summary: generative AI hub SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/sap-genai-hub/lib/python3.11/site-packages
Requires: ai-core-sdk, click, dacite, openai, overloading, packaging, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


Run the code cell below to install SAP AI Core SDK, if it's missing.

In [2]:
try:
    from ai_core_sdk.ai_core_v2_client import AICoreV2Client
    %pip show ai-core-sdk
except:
    %pip install ai-core-sdk

Name: ai-core-sdk
Version: 2.4.12
Summary: SAP AI Core SDK
Home-page: https://www.sap.com/
Author: SAP SE
Author-email: 
License: SAP DEVELOPER LICENSE AGREEMENT
Location: /opt/homebrew/Caskroom/miniconda/base/envs/sap-genai-hub/lib/python3.11/site-packages
Requires: ai-api-client-sdk, click
Required-by: generative-ai-hub-sdk
Note: you may need to restart the kernel to use updated packages.


Download the Service Key for your SAP AI Core instance, save it in the same directory as this Jupyter notebook, and name it as `aicore_service_key.json`. The content of the file should look like this:

```json
{
    "clientid": "sb-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx!xxxxxxx|aicore!xxxx",
    "clientsecret": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx$***",
    "url": "https://***.authentication.eu10.hana.ondemand.com",
    "identityzone": "***",
    "identityzoneid": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx",
    "appname": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx!xxxxxxx|aicore!xxxx",
    "serviceurls": {
        "AI_API_URL": "https://api.ai.***.hana.ondemand.com"
    }
}
```

Create a file named `config.json` in the same directory as this Jupyter notebook. The content of the file should look like this:

```json
{
    "AICORE_AUTH_URL": "https://***.authentication.eu10.hana.ondemand.com",
    "AICORE_CLIENT_ID": "sb-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx!xxxxxxx|aicore!xxxx",
    "AICORE_CLIENT_SECRET": "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx$***",
    "AICORE_RESOURCE_GROUP": "***",
    "AICORE_BASE_URL": "https://api.ai.***.hana.ondemand.com/v2"
}

```

## Connect to SAP AI Core

In [3]:
import json

# Your service key JSON file relative to this notebook
aicore_service_key_path = 'aicore_service_key.json'

# Load the service key file
with open(aicore_service_key_path) as ask:
    aicore_service_key = json.load(ask)

In [4]:

from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Create an AI API client instance
ai_core_client = AICoreV2Client(
    base_url = aicore_service_key["serviceurls"]["AI_API_URL"] + "/v2",
    auth_url=  aicore_service_key["url"] + "/oauth/token",
    client_id = aicore_service_key['clientid'],
    client_secret = aicore_service_key['clientsecret']
)

# Get the number of GitHub repositories connected to SAP AI Core
response = ai_core_client.repositories.query()
print(response.count)

5


## Create a configuration

In [5]:
from ai_core_sdk.models import ParameterBinding

response = ai_core_client.configuration.create(
    name = "embedding-model-demo-config",
    resource_group = "genai-demo",
    scenario_id = "foundation-models",
	executable_id = "azure-openai",
    parameter_bindings = [
        ParameterBinding(key="modelName", value="text-embedding-ada-002"),
        ParameterBinding(key="modelVersion", value="2"),
    ]
)

print(response.__dict__)
configuration_id = response.__dict__['id']

{'id': 'b7d76007-83df-45c2-a1a3-c8a20042810d', 'message': 'Configuration created'}


## Create a deployment

In [6]:
response = ai_core_client.deployment.create(
    resource_group = "genai-demo",
    configuration_id = configuration_id
)

print(response.__dict__)
deployment_id=response.__dict__['id']

{'id': 'd6e4a354ea8e417c', 'message': 'Deployment scheduled.', 'deployment_url': '', 'status': <Status.UNKNOWN: 'UNKNOWN'>, 'ttl': None}


In [7]:
import time

while str(response.status)!='Status.RUNNING':

    response = ai_core_client.deployment.get(
        resource_group = 'genai-demo',
        deployment_id = deployment_id
    )

    print("Status: ", response.status)
    print('*'*80)
    time.sleep(30)

print(response.__dict__)
deployment_url = response.__dict__['deployment_url']

Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.UNKNOWN
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.PENDING
********************************************************************************
Status:  Status.RUNNING
********************************************************************************
{'id': 'd6e4a354ea8e417c', 'configuration_id': 'b7d7600

## Access the Embedding Model
SAP Generative AI Hub SDK offers a developer-friendly way to consume foundational models available in SAP Generative AI Hub. It provides integrations that act as drop-in replacements for native client SDKs and LangChain, allowing developers to use familiar interfaces and workflows. The samples bellow demonstrate how to use SAP Generative AI Hub SDK to make API calls to the deployed embedding model to generate embeddings from text input.

By default, SAP Generative AI Hub SDK loads SAP AI Core credentials and configuration settings as environment variables, from a configuration file `~/.aicore/config.json`. With the following code cell, it would load from `./config.json` instead.

In [8]:
import os
os.environ["AICORE_HOME"] = os.getcwd()

### LangChain Integraion
LangChain provides an interface that abstracts provider-specific details into a common interface. Harmonized model initialization with `init_embedding_model` function allows easy initialization of langchain model interface.

In [9]:
from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

text = 'Every decoding is another encoding.'

embeddings = init_embedding_model('text-embedding-ada-002')
response = embeddings.embed_query(text)
print(response)

[-0.013148151338100433, 0.0028077815659344196, 0.02173510380089283, -0.019712312147021294, 0.0054007237777113914, -0.006494750734418631, -0.005245378706604242, -0.011171632446348667, -0.005446997005492449, -0.03183584660291672, 0.005820486228913069, 0.02615087293088436, 0.00936037302017212, -0.0058799805119633675, -0.01532298419624567, -0.021854091435670853, 0.019276022911071777, 0.010338717140257359, 0.022065626457333565, -0.03165075555443764, -0.03183584660291672, 0.010523810051381588, -0.007992013357579708, -0.032285355031490326, -0.01742510125041008, 0.007892857305705547, 0.003761336440220475, -0.01751764863729477, 0.0013510072603821754, -0.007059942465275526, 0.00856712181121111, -0.01199793629348278, 0.0018492686795070767, -0.02784314565360546, -0.013881909660995007, -0.021576454862952232, -0.011885559186339378, -0.03403051197528839, 0.0055329324677586555, -0.004861973691731691, -0.0008308488177135587, -0.005496575031429529, -0.011601310223340988, -0.002042623935267329, -0.022541

### Native Client Integrations
Currently, integrations are available for three native client SDKs (OpenAI, Google, Amazon). The code snippet bellow shows how to use `embeddings` in Generative AI Hub SDK, which is equivalent to `openai.embeddings`.

In [10]:
from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="Every decoding is another encoding.",
    model_name="text-embedding-ada-002"
)

print(response.data)

[Embedding(embedding=[-0.013148151338100433, 0.0028077815659344196, 0.02173510380089283, -0.019712312147021294, 0.0054007237777113914, -0.006494750734418631, -0.005245378706604242, -0.011171632446348667, -0.005446997005492449, -0.03183584660291672, 0.005820486228913069, 0.02615087293088436, 0.00936037302017212, -0.0058799805119633675, -0.01532298419624567, -0.021854091435670853, 0.019276022911071777, 0.010338717140257359, 0.022065626457333565, -0.03165075555443764, -0.03183584660291672, 0.010523810051381588, -0.007992013357579708, -0.032285355031490326, -0.01742510125041008, 0.007892857305705547, 0.003761336440220475, -0.01751764863729477, 0.0013510072603821754, -0.007059942465275526, 0.00856712181121111, -0.01199793629348278, 0.0018492686795070767, -0.02784314565360546, -0.013881909660995007, -0.021576454862952232, -0.011885559186339378, -0.03403051197528839, 0.0055329324677586555, -0.004861973691731691, -0.0008308488177135587, -0.005496575031429529, -0.011601310223340988, -0.00204262